In [ ]:
import scanpy as sc
import scipy.sparse as sp

# Read h5ad file
print("Reading h5ad file...")
adata = sc.read_h5ad('../scRNA_Project/processed_data/strict_epdsc_annotated_data.h5')

print("Converting X to CSC sparse matrix...")
# Convert main matrix to CSC (fast for gene queries)
if sp.issparse(adata.X):
    adata.X = adata.X.tocsc()

print("Converting layers to CSC sparse matrices...")
# Convert all layers to CSC if they are sparse
for layer in adata.layers:
    if sp.issparse(adata.layers[layer]):
        adata.layers[layer] = adata.layers[layer].tocsc()
    else:
        adata.layers[layer] = sp.csc_matrix(adata.layers[layer])

print("Creating Zarr store...")
# Save as Zarr
adata.write_zarr('../scRNA_Project/processed_data/strict_epdsc_annotated_data_full_csc.zarr')


Reading h5ad file...
Converting X to CSC sparse matrix...
Converting layers to CSC sparse matrices...
Creating Zarr store...


In [30]:
import scanpy as sc
import scipy.sparse as sp
import anndata as ad

# Read h5ad file
print("Reading h5ad file...")
adata = ad.experimental.read_lazy('https://rubioinformatics.s3.us-east-1.amazonaws.com/Test_zarr/corrected_doublet_decontX_mt_Duo_allergy.zarr')
# adata = ad.experimental.read_lazy('https://storage.googleapis.com/scrna-seqbrowser/BoneMarrow_May2024_outliers_removed_Palantir_csc.zarr')

Reading h5ad file...


In [39]:
import zarr
import numpy as np

def get_obs_column_fast(z, name, cells=None):

    # Check root keys
    if "obs" not in z.keys():
        return None

    obs = z["obs"]
    if name not in obs.keys():
        return None

    col_obj = obs[name]

    # Detect if it's a zarr.Group representing a categorical column
    if isinstance(col_obj, zarr.Group):
        full_codes = col_obj["codes"][:]
        full_categories = col_obj["categories"][:]
        # Convert to Python list or NumPy array of categories
        full_col = np.array([full_categories[i] for i in full_codes])
    else:
        full_col = col_obj[:]

    # Handle subsetting
    if cells is None or cells == "all":
        return full_col
    elif isinstance(cells, (list, np.ndarray)) and all(1 <= c <= len(full_col) for c in cells):
        # Convert to 0-based indexing
        return full_col[np.array(cells) - 1]
    else:
        raise ValueError("Invalid cell indices provided")

z = zarr.open('https://rubioinformatics.s3.us-east-1.amazonaws.com/Test_zarr/corrected_doublet_decontX_mt_Duo_allergy.zarr', mode="r")

get_obs_column_fast(z, "cyclon_Phase")

array(['G1', 'G1', 'G1', ..., 'G1', 'G1', 'G1'],
      shape=(2079,), dtype='<U3')

In [21]:
sc.tl.rank_genes_groups(adata, 'Annotation')

AttributeError: 'DataArray' object has no attribute 'cat'